In [1]:
import os
import time

import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v4"
EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']


def focal_loss(alpha=0.25, gamma=1.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
        return loss

    return loss


optimizers = {
    "Adam": lambda lr: tf.keras.optimizers.Adam(learning_rate=lr),
    "SGD": lambda lr: tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9),
    "RMSprop": lambda lr: tf.keras.optimizers.RMSprop(learning_rate=lr)
}

learning_rates = [0.01, 0.001, 0.0005, 0.0001]


def train_and_evaluate(optimizer_name, learning_rate, VERSION_INDEX):
    loss_name = f"{optimizer_name}_LR{learning_rate}"
    print(f"\n🔹 Entrenando con {optimizer_name} y learning rate {learning_rate} 🔹")

    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}.{VERSION_INDEX}")

    model.compile(
        optimizer=optimizers[optimizer_name](learning_rate),
        loss=focal_loss(alpha=0.25, gamma=1.0),
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}", ),
        ]
    )
    training_time = time.time() - start_time

    results = model.evaluate(test_images)

    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}", exist_ok=True)
    history_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/history_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["optimizer"] = optimizer_name
    evaluation_results["learning_rate"] = learning_rate
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(
        f"./logs/{MODEL_NAME}/{VERSION_BASE}/{VERSION_BASE}.{VERSION_INDEX}/results_{VERSION_BASE}.{VERSION_INDEX}.csv",
        index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


count = 0
all_results = []
for optimizer_name in optimizers.keys():
    for learning_rate in learning_rates:
        results_df = train_and_evaluate(optimizer_name, learning_rate, count)
        all_results.append(results_df)
        count += 1

final_results = pd.concat(all_results, ignore_index=True)
final_results.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/optimizer_tuning_results.csv", index=False)

print("\n✅ ¡Experimento con optimización de optimizadores y learning rate completado!")


🔹 Entrenando con Adam y learning rate 0.01 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


2025-02-23 21:26:07.502482: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-23 21:26:07.502512: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-23 21:26:07.502516: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-23 21:26:07.502744: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-23 21:26:07.502760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2025-02-23 21:26:09.028126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0395 - accuracy: 0.8941 - precision: 0.9298 - recall: 0.9339 - auc: 0.9444

...

Epoch 1/10


2025-02-23 23:55:03.866148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0452 - accuracy: 0.8316 - precision: 0.8546 - recall: 0.9432 - auc: 0.8713

2025-02-23 23:56:15.659976: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - 91s 169ms/step - loss: 0.0452 - accuracy: 0.8316 - precision: 0.8546 - recall: 0.9432 - auc: 0.8713 - val_loss: 0.0351 - val_accuracy: 0.8950 - val_precision: 0.8924 - val_recall: 0.9566 - val_auc: 0.9570
Epoch 2/10
533/533 [==============================] - 89s 168ms/step - loss: 0.0322 - accuracy: 0.8879 - precision: 0.9125 - recall: 0.9462 - auc: 0.9409 - val_loss: 0.0283 - val_accuracy: 0.9095 - val_precision: 0.9203 - val_recall: 0.9450 - val_auc: 0.9691
Epoch 3/10
533/533 [==============================] - 90s 170ms/step - loss: 0.0291 - accuracy: 0.9020 - precision: 0.9268 - recall: 0.9485 - auc: 0.9526 - val_loss: 0.0261 - val_accuracy: 0.9151 - val_precision: 0.9402 - val_recall: 0.9309 - val_auc: 0.9728
Epoch 4/10
533/533 [==============================] - 91s 171ms/step - loss: 0.0275 - accuracy: 0.9084 - precision: 0.9333 - recall: 0.9497 - auc: 0.9581 - val_loss: 0.0247 - val_accuracy: 0.9200 - val_precision: 0.9409 - val_recall: 